In [49]:
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
user_item_matrix = np.array([
    [5,3,0,1],
    [4,0,4,0],
    [0,2,1,4],
    [1,0,5,4],
])

user_similarity = cosine_similarity(user_item_matrix)
def get_user_based_reccommendations(user_id,user_similarity,user_item_matrix,num_reccommendations=3):
    user_similarities = user_similarity[user_id]
    similar_users_sorted = np.argsort(user_similarities[::-1])
    unrated_items = np.where(user_item_matrix[user_id]==0)[0]
    reccommendations = []
    for user in similar_users_sorted:
        for item in unrated_items:
            if user_item_matrix[user,item] > 0:
                reccommendations.append((item, user_item_matrix[user,item]))
    reccommendations.sort(key=lambda x: x[1], reverse = True)
    recommended_items = [item for item, _ in reccommendations[:num_reccommendations]]
    return recommended_items
target_user_id = 0
reccommendations = get_user_based_reccommendations(target_user_id, user_similarity, user_item_matrix)
print("Reccommended items for the target user : ")
for item_id in reccommendations:
    print(f"Item {item_id}")

Reccommended items for the target user : 
Item 2
Item 2
Item 2


<h1> item based coil filter </h1>

In [51]:
item_similarity = cosine_similarity(user_item_matrix.T)
def get_user_based_reccommendations(user_id,item_similarity,user_item_matrix,num_reccommendations=3):
    user_similarities = item_similarity[user_id]
    similar_users_sorted = np.argsort(user_similarities[::-1])
    unrated_items = np.where(user_item_matrix[user_id]==0)[0]
    reccommendations = []
    for user in similar_users_sorted:
        for item in unrated_items:
            if user_item_matrix[user,item] > 0:
                reccommendations.append((item, user_item_matrix[user,item]))
    reccommendations.sort(key=lambda x: x[1], reverse = True)
    recommended_items = [item for item, _ in reccommendations[:num_reccommendations]]
    return recommended_items
target_user_id = 0
reccommendations = get_user_based_reccommendations(target_user_id, item_similarity, user_item_matrix)
print("Reccommended items for the target user : ")
for item_id in reccommendations:
    print(f"Item {item_id}")

Reccommended items for the target user : 
Item 2
Item 2
Item 2


In [52]:
from scipy.sparse.linalg import svds
matrix = np.array(user_item_matrix, float)
u,sigma, Vt = svds(matrix, k=2)
sigma_diag = np.diag(sigma)
predicted_user_item_matrix = np.dot(np.dot(u,sigma_diag),Vt)
def get_model_based_reccommendations(user_id, predicted_user_item_matrix, num_reccommendations=3):
    predicted_interactions = predicted_user_item_matrix[user_id]
    unrated_items = np.where(predicted_interactions == 0)[0]
    sorted_unrated_items = np.argsort(predicted_interactions)[::-1]
    recommended_items = [item for item in sorted_unrated_items[:num_reccommendations]]
    return recommended_items
target_user_id = 0
reccommendations = get_model_based_reccommendations(target_user_id, predicted_user_item_matrix)
print("Reccommended items for the target user : ")
for item_id in reccommendations:
    print(f"Item {item_id}")


Reccommended items for the target user : 
Item 0
Item 1
Item 2


In [53]:
user_similarity

array([[1.        , 0.5976143 , 0.36885556, 0.23473824],
       [0.5976143 , 1.        , 0.15430335, 0.65465367],
       [0.36885556, 0.15430335, 1.        , 0.70710678],
       [0.23473824, 0.65465367, 0.70710678, 1.        ]])

In [54]:
item_similarity

array([[1.        , 0.64194074, 0.5       , 0.24174689],
       [0.64194074, 1.        , 0.0855921 , 0.531085  ],
       [0.5       , 0.0855921 , 1.        , 0.64465837],
       [0.24174689, 0.531085  , 0.64465837, 1.        ]])

In [55]:
predicted_user_item_matrix

array([[ 5.15341682,  1.94976022,  1.07587267, -0.09126383],
       [ 3.7873761 ,  1.47704855,  2.47308024,  1.5491318 ],
       [ 0.32711665,  0.18960034,  2.57880948,  2.4059535 ],
       [ 0.89706489,  0.45748848,  4.69026123,  4.30993496]])